In [1]:
from pymanopt.manifolds import Grassmann, ComplexGrassmann, SpecialOrthogonalGroup, Euclidean
import numpy as np
from compute_centroid import *
from grass_DR import *
from PNG import *
from sklearn.decomposition import PCA
from PGA import *
from Complex_PCA import *
import matplotlib.pyplot as plt

# Example 1: 15 points on Gr(2, 6) embedded in Gr(2, 10) 

## Generate data

In [2]:
m = 6
n = 10
p = 2
N = 15
sig = 0
gr_low = Grassmann(m, p, N)
gr = Grassmann(n, p, N)
gr_map = Grassmann(n, m)

X_low = gr_low.rand() # N x m x p
A = gr_map.rand() # n x m
#B = np.random.normal(0, 0.1, (n, p)) # n x p
B = np.zeros((n,p))
AAT = np.matmul(A, A.T) 
IAATB = np.matmul(np.eye(n) - AAT, B)
X = np.array([np.linalg.qr(np.matmul(A, X_low[i]) + IAATB)[0] for i in range(N)]) # N x n x p
X = gr.exp(X, sig * gr.randvec(X)) # perturb the emdedded X

## Compute the score using PNG and apply PCA to the scores

In [3]:
scores_PNG = PNG(X, verbosity = 1)

Gr(2, 10) -> Gr(2, 9)
Terminated - min grad norm reached after 130 iterations, 0.79 seconds.

Gr(2, 9) -> Gr(2, 8)
Terminated - min grad norm reached after 70 iterations, 0.41 seconds.

Gr(2, 8) -> Gr(2, 7)
Terminated - min grad norm reached after 49 iterations, 0.29 seconds.

Gr(2, 7) -> Gr(2, 6)
Terminated - min grad norm reached after 42 iterations, 0.26 seconds.

Gr(2, 6) -> Gr(2, 5)
Terminated - min grad norm reached after 512 iterations, 2.78 seconds.

Gr(2, 5) -> Gr(2, 4)
Terminated - min grad norm reached after 223 iterations, 1.21 seconds.

Gr(2, 4) -> Gr(2, 3)
Terminated - max iterations reached after 5.53 seconds.

Gr(2, 3) -> Gr(1, 3)
Gr(1, 3) -> Gr(1, 2)
Terminated - min grad norm reached after 37 iterations, 0.23 seconds.

Gr(1, 2) -> NGM


In [4]:
n_c = 5
pca = PCA(n_components = n_c)
pca.fit(scores_PNG)
print(f'The ratios of expressed variance of the first {n_c} PCs.')
print(pca.explained_variance_ratio_.round(2))
print(f'The cumulative ratios of expressed variance of the first {n_c} PCs.')
print(np.cumsum(pca.explained_variance_ratio_).round(2))

The ratios of expressed variance of the first 5 PCs.
[0.47 0.23 0.11 0.09 0.07]
The cumulative ratios of expressed variance of the first 5 PCs.
[0.47 0.71 0.81 0.9  0.97]


## PGA

In [5]:
gr = Grassmann(n, p)
n_c = 5
pga = PGA(X, n_c, gr)

print(f'The ratios of expressed variance of the first {n_c} PCs.')
print(pga.explained_variance_ratio_.round(2))
print(f'The cumulative ratios of expressed variance of the first {n_c} PCs.')
print(np.cumsum(pga.explained_variance_ratio_).round(2))

The ratios of expressed variance of the first 5 PCs.
[0.26 0.25 0.16 0.12 0.07]
The cumulative ratios of expressed variance of the first 5 PCs.
[0.26 0.51 0.67 0.79 0.86]


# Example 2: 15 points on Gr(2, $C^4$) embedded in Gr(2, $C^5$) 

## Generate data

In [12]:
m = 4
n = 5
p = 2
N = 15
sig = 0.1
cgr_low = ComplexGrassmann(m, p, N)
cgr = ComplexGrassmann(n, p, N)
cgr_map = ComplexGrassmann(n, m)

X_low = cgr_low.rand() # N x m x p
A = cgr_map.rand() # n x m
#B = np.random.normal(0, 0.1, (n, p)) # n x p
B = np.zeros((n,p))
AAT = np.matmul(A, A.conj().T) 
IAATB = np.matmul(np.eye(n) - AAT, B)
X = np.array([np.linalg.qr(np.matmul(A, X_low[i]) + IAATB)[0] for i in range(N)]) # N x n x p
X = cgr.exp(X, sig * cgr.randvec(X)) # perturb the emdedded X

In [15]:
scores_PNG = PNG(X, verbosity = 1)

Gr(2, 5) -> Gr(2, 4)
Terminated - min grad norm reached after 290 iterations, 2.25 seconds.

Gr(2, 4) -> Gr(2, 3)
Terminated - min grad norm reached after 90 iterations, 0.71 seconds.

Gr(2, 3) -> Gr(1, 3)
Gr(1, 3) -> Gr(1, 2)
Terminated - min grad norm reached after 79 iterations, 0.59 seconds.

Gr(1, 2) -> NGM


In [16]:
n_c = 5
cpca = Complex_PCA(n_components = n_c)
cpca.fit(scores_PNG)
print(f'The ratios of expressed variance of the first {n_c} PCs.')
print(cpca.explained_variance_ratio_.round(2))
print(f'The cumulative ratios of expressed variance of the first {n_c} PCs.')
print(np.cumsum(cpca.explained_variance_ratio_).round(2))

The ratios of expressed variance of the first 5 PCs.
[0.77 0.1  0.07 0.04 0.01]
The cumulative ratios of expressed variance of the first 5 PCs.
[0.77 0.88 0.95 0.98 1.  ]


## PGA

In [17]:
N, n, p = X.shape
man = ComplexGrassmann(n, p)
cpga = Complex_PGA(X, n_c, man)
print(f'The ratios of expressed variance of the first {n_c} PCs.')
print(cpga.explained_variance_ratio_.round(2))
print(f'The cumulative ratios of expressed variance of the first {n_c} PCs.')
print(np.cumsum(cpga.explained_variance_ratio_).round(2))

The ratios of expressed variance of the first 5 PCs.
[0.5  0.22 0.18 0.1  0.  ]
The cumulative ratios of expressed variance of the first 5 PCs.
[0.5  0.73 0.9  1.   1.  ]


# Example 3: Supervised Principal Nested Grassmann

## Generate Data

In [22]:
m = 2
n = 5
p = 1
N = 50
n_class = 2
sig = 0.1

gr_low = Grassmann(m, p, N)
y = np.random.randint(n_class, size = N)
M = np.array([[[1],[0]],[[0],[1]]])
X_low = np.zeros((N, m, p))
# translate the X such that different classes are separated
gr = Grassmann(m, p)
for i in range(N):
    X_low[i] = gr.exp(M[y[i]], 0.5 * gr.randvec(M[y[i]]))

gr_map = Grassmann(n, m)
A = gr_map.rand()
#B = np.random.normal(0, 0.1, (n, p)) # n_class x n x p
B = np.zeros((n, p))
AAT = np.matmul(A, A.T) 
IAATB = np.matmul(np.eye(n) - AAT, B)


gr = Grassmann(n, p, N)

X = np.array([np.linalg.qr(np.matmul(A, X_low[i]) + IAATB)[0] for i in range(N)]) # N x n x p
X = gr.exp(X, sig * gr.randvec(X)) # perturb the emdedded X

## Compute the score using SPNG

In [23]:
scores_PNG = SPNG(X, y)

Gr(1, 5) -> Gr(1, 4)
Gr(1, 4) -> Gr(1, 3)
Gr(1, 3) -> Gr(1, 2)
Gr(1, 2) -> NGM
